In [ ]:
from __future__ import division, print_function
import sys
import os
import glob
import re
import numpy as np
from matplotlib.image import imread
import cv2

# Keras
from keras.models import model_from_json

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer

# Define a flask app
app = Flask(__name__)

@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')

json_file = open("models/test_model.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("models/test_model.h5")

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Model loaded. Check http://127.0.0.1:5000/')

@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        file_path = os.path.join('./uploads', secure_filename(f.filename))
        f.save(file_path)
        
       

        # Make prediction
        preds = model_predict(file_path, model)
        alp=['Test Negative','Test Positive']
        print(str(preds))
        if preds[0][0]>preds[0][1]:
            return alp[0]
        else:
            return alp[1]
    return None

def model_predict(img_path, model):
    img = imread(img_path)
    x=cv2.resize(img,(48,48))
    x = np.expand_dims(x, axis=0)
    x = np.expand_dims(x, axis=-1)
   
    preds = model.predict(x)
    return preds


if __name__ == '__main__':
    app.run(debug=False)

Model loaded. Check http://127.0.0.1:5000/
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Apr/2022 20:06:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 20:06:50] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Apr/2022 20:07:16] "POST /predict HTTP/1.1" 200 -


[[0.03798235 0.9620176 ]]


[2022-04-22 20:07:43,948] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "C:\Users\arkap\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\arkap\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\arkap\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\arkap\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\arkap\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\arkap\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\arkap\AppData\Local\Temp/ipykernel_31800/3416284617.py", line 46,

[[0.00204188 0.99795806]]
